In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv

In [13]:
# Clean dataset, take only necessary columns, create new dataframes and save as csv

# File to Load
fastfood_to_load = "datasets/fast-food-restaurants/Datafiniti_Fast_Food_Restaurants.csv"
census_to_load = "datasets/us-census-demographic-data/acs2017_census_tract_data.csv"

# Read the CSV files
fastfood = pd.read_csv(fastfood_to_load)
census = pd.read_csv(census_to_load)

# Create FastFood dataset
ff_id = fastfood['id']
ff_name = fastfood['name']
ff_state = fastfood['province']
dict1 = {'state': ff_state, 'name': ff_name}  
fastfood_df = pd.DataFrame(dict1) 

# Output File (CSV)
output_ff_file = "output_data/fastfood.csv"

# Write DF to a CSV
fastfood_df.to_csv(output_ff_file, sep=';', encoding='utf-8')


# Create Census dataset
# Columns: State, Income, Professional, Unemployment
us_id = census['TractId']
us_state = census['State']
us_income = census['Income']
us_professional = census['Professional']
us_unemployment = census['Unemployment']
dict2 = {'state': us_state, 'income': us_income, 'professional': us_professional, 'unemployment': us_unemployment}
census_df = pd.DataFrame(dict2)

# Output File (CSV)
output_us_file = "output_data/census.csv"

# Write output to the file
census_df.to_csv(output_us_file, sep=';', encoding='utf-8')


In [14]:
# This is required to change state names into abbreviations in order to merge datasets later
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [15]:
fastfood_df.head()

,state,name
0,LA,SONIC Drive In
1,LA,SONIC Drive In
2,TN,Taco Bell
3,TN,Arby's
4,GA,Steak 'n Shake


In [16]:
# Change state abbreviation to state name
census_df = census_df.replace({"state": us_state_abbrev})

In [17]:
census_df.head()

,state,income,professional,unemployment
0,AL,67826.0,38.5,4.6
1,AL,41287.0,30.5,3.4
2,AL,46806.0,27.9,4.7
3,AL,55895.0,29.0,6.1
4,AL,68143.0,48.8,2.3


## Questions:
1. most popular FFR by state
2. correlation between white collar jobs and FFR
3. correlation between unemployment rate and FFR
4. correlation between income and FFR

In [26]:
# Clean NaNs from datasets
census_df = census_df.dropna()
fastfood_df = fastfood_df.dropna()

In [27]:
#ffbs_df most popular fastfood in each state. 
ffbs_df = fastfood_df.groupby('state', as_index=False).agg({'name': lambda x: x.value_counts().index[0]})
ffbs_df.columns = ['state', 'most_popular_fastfood']
ffbs_df.head()

,state,most_popular_fastfood
0,AK,Subway
1,AL,Taco Bell
2,AR,McDonald's
3,AZ,McDonald's
4,CA,McDonald's


In [28]:
# number of FFRs in each state
ff_qty_df = fastfood_df.groupby('state', as_index=False).count()
ff_qty_df.columns = ['state', 'fastfood_qty']
ff_qty_df.head()

,state,fastfood_qty
0,AK,16
1,AL,6
2,AR,102
3,AZ,330
4,CA,1201


In [29]:
# Average income, number of prefessionals, and unemployed in each state
cen_mean_df = census_df.groupby('state', as_index=False).mean()
cen_mean_df.head()

,state,income,professional,unemployment
0,AK,73796.757576,35.360606,8.700606
1,AL,45938.212947,30.788501,8.644549
2,AR,44245.267936,30.750073,7.060322
3,AZ,57815.571807,34.612773,7.580278
4,CA,73070.965821,37.049925,7.856258


In [32]:
# Merging datasets to create final DS ready for plotting.
cen_mpf_df = pd.merge(cen_mean_df, ffbs_df, on='state', how='outer')
df = pd.merge(cen_mpf_df, ff_qty_df, on='state', how='outer')
df = df.dropna()
df

,state,income,professional,unemployment,most_popular_fastfood,fastfood_qty
0,AK,73796.757576,35.360606,8.700606,Subway,16.0
1,AL,45938.212947,30.788501,8.644549,Taco Bell,6.0
2,AR,44245.267936,30.750073,7.060322,McDonald's,102.0
3,AZ,57815.571807,34.612773,7.580278,McDonald's,330.0
4,CA,73070.965821,37.049925,7.856258,McDonald's,1201.0
5,CO,69595.696429,40.240097,5.487338,Taco Bell,148.0
6,CT,79622.418919,41.235012,7.814496,Subway,53.0
7,DE,64310.537383,39.043925,6.601869,Arby's,44.0
9,FL,54263.602892,34.210458,7.425759,McDonald's,621.0
10,GA,54650.184426,34.375666,8.185143,McDonald's,420.0
